In [ ]:
# ============================================================
# セル1: 環境セットアップ（1回実行）
# ============================================================

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import json
import warnings
warnings.filterwarnings('ignore')

# 設定
DB_PATH = 'pachinko_analysis_マルハンメガシティ柏.db'
plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# イベント定義（グローバル）
EVENT_DEFINITIONS = {
    'is_1day': '1day', 'is_2day': '2day', 'is_3day': '3day',
    'is_4day': '4day', 'is_5day': '5day', 'is_6day': '6day',
    'is_7day': '7day', 'is_8day': '8day', 'is_9day': '9day',
    'is_0day': '0day', 'is_39day': '39day', 'is_40day': '40day',
    'is_zorome': 'Zorome', 'is_saturday': 'Saturday', 'is_sunday': 'Sunday'
}

DIGIT_ORDER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ゾロ目']

print("✅ 環境セットアップ完了")

In [ ]:
# ============================================================
# セル2: データ読み込み（1回実行）
# ============================================================

def load_and_prepare_data(db_path):
    """全データ読込・整形"""
    conn = sqlite3.connect(db_path)
    
    # 3テーブル読込
    tables = {
        'all': 'last_digit_summary_all',
        'jug': 'last_digit_summary_jug',
        'non_jug': 'last_digit_summary_other'
    }
    
    df_events = pd.read_sql_query("SELECT * FROM event_calendar ORDER BY date", conn)
    data = {}
    
    for key, table in tables.items():
        try:
            df = pd.read_sql_query(f"SELECT * FROM {table} ORDER BY date, last_digit", conn)
            df = df.merge(df_events, on='date', how='left')
            
            # 日付処理
            df['date_obj'] = pd.to_datetime(df['date'], format='%Y%m%d')
            df['weekday'] = df['date_obj'].dt.day_name()
            df['weekday_num'] = df['date_obj'].dt.weekday
            df['is_saturday'] = (df['weekday_num'] == 5).astype(int)
            df['is_sunday'] = (df['weekday_num'] == 6).astype(int)
            df['last_digit'] = pd.Categorical(df['last_digit'], categories=DIGIT_ORDER, ordered=True)
            
            data[key] = df
            print(f"{key}: {len(df)} rows, {df['date'].min()} - {df['date'].max()} ({df['date'].nunique()} days)")
        except:
            data[key] = pd.DataFrame()
    
    conn.close()
    return data['all'], data['jug'], data['non_jug']

df_all, df_jug, df_non_jug = load_and_prepare_data(DB_PATH)

print("\n✅ データ読み込み完了")

In [ ]:
# ============================================================
# セル3: 共通関数定義（1回実行）
# ============================================================

def create_event_history_features(df):
    """イベント履歴特徴量作成（インデックス化版）"""
    all_dates = sorted(df['date'].unique())
    
    # インデックス構築: {(event_type, digit): [date1, date2, ...]}
    print("  インデックス構築中...")
    event_digit_index = {}
    
    for date in all_dates:
        date_mask = df['date'] == date
        date_data = df[date_mask].iloc[0]
        
        # アクティブイベント特定
        active_events = []
        for col, label in EVENT_DEFINITIONS.items():
            if col in df.columns:  # ← DataFrameのカラムで確認
                if date_data.get(col, 0) == 1:
                    active_events.append(label.lower())
        
        # 各末尾×イベントの組み合わせにdateを追加
        for digit in df[date_mask]['last_digit'].unique():
            for event_name in active_events:
                key = (event_name, digit)
                if key not in event_digit_index:
                    event_digit_index[key] = []
                event_digit_index[key].append(date)
    
    # 差枚キャッシュ: {(date, digit): diff}
    diff_cache = df.set_index(['date', 'last_digit'])['avg_diff_coins'].to_dict()
    
    # 特徴量生成
    print("  特徴量生成中...")
    history_features = []
    
    for (event_name, digit), dates in event_digit_index.items():
        for i, date in enumerate(dates):
            current_diff = diff_cache.get((date, digit))
            if current_diff is None:
                continue
            
            # 過去の差枚を直接取得
            prev_1 = diff_cache.get((dates[i-1], digit)) if i >= 1 else None
            prev_2 = diff_cache.get((dates[i-2], digit)) if i >= 2 else None
            
            history_features.append({
                'date': date, 'last_digit': digit, 'event_type': event_name,
                'current_diff': current_diff, 'prev_1_diff': prev_1, 'prev_2_diff': prev_2,
                'prev_1_change': current_diff - prev_1 if prev_1 else None,
                'prev_2_change': current_diff - prev_2 if prev_2 else None,
            })
    
    return pd.DataFrame(history_features)

def prepare_enhanced_features(history_df, event_type, cutoff_date, include_current=False):
    """機械学習用特徴量準備
    
    Args:
        include_current: Trueの場合cutoff_dateを含む（学習時）、Falseで除外（予測時）
    """
    if include_current:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] <= cutoff_date)
        ].copy()
    else:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] < cutoff_date)
        ].copy()
    
    if len(event_data) == 0:
        return None, None, None
    
    X = event_data[['prev_1_diff', 'prev_2_diff', 'prev_1_change', 'prev_2_change']].fillna(0)
    y = (event_data['current_diff'] > event_data['current_diff'].median()).astype(int)
    
    return X, y, event_data

def analyze_trend(df, recent_days=30):
    """全期間 vs 直近トレンド分析"""
    latest_date = sorted(df['date'].unique())[-1]
    cutoff_date = (pd.to_datetime(latest_date, format='%Y%m%d') - timedelta(days=recent_days)).strftime('%Y%m%d')
    
    trends = {}
    for col, label in EVENT_DEFINITIONS.items():
        if col not in df.columns:
            continue
        
        event_data = df[df[col] == 1] if col.startswith('is_') else df[df['weekday_num'] == (5 if 'saturday' in col else 6)]
        recent_data = event_data[event_data['date'] >= cutoff_date]
        
        trends[label.lower()] = {}
        for digit in df['last_digit'].unique():
            all_avg = event_data[event_data['last_digit'] == digit]['avg_diff_coins'].mean()
            recent_avg = recent_data[recent_data['last_digit'] == digit]['avg_diff_coins'].mean()
            
            if pd.notna(all_avg) and pd.notna(recent_avg):
                trends[label.lower()][str(digit)] = {
                    'all_period_avg': round(all_avg, 1),
                    'recent_avg': round(recent_avg, 1),
                    'trend': round(recent_avg - all_avg, 1)
                }
    
    return trends

print("✅ 共通関数定義完了")

In [ ]:
# ============================================================
# セル4: イベント履歴・トレンド分析実行（1回実行）
# ============================================================

# 3モデル分の履歴作成（df_all, df_jug, df_non_jugを直接使用）
histories = {}
trends = {}

for name, df in [('all', df_all), ('jug', df_jug), ('non_jug', df_non_jug)]:
    if len(df) > 0:
        print(f"【{name.upper()}】履歴作成中...")
        histories[name] = create_event_history_features(df)
        print(f"  {len(histories[name])} records, 欠損率: prev_1={histories[name]['prev_1_diff'].isna().mean():.1%}")
        
        trends[name] = analyze_trend(df, recent_days=30)

print("\n✅ 履歴・トレンド分析完了")

In [ ]:
# ============================================================
# TOP1/TOP2取得回数ヒートマップ
# ============================================================

def plot_top_rank_heatmap(df, title, rank_type='top1'):
    """
    TOP1またはTOP2を取った回数のヒートマップ
    
    Parameters:
    -----------
    df : DataFrame
        分析対象データ
    title : str
        タイトル
    rank_type : str
        'top1' or 'top2'
    """
    df = df.copy()
    
    # イベント×末尾でTOPランク取得回数を集計
    pivot_data = []
    
    for col, label in EVENT_DEFINITIONS.items():
        if col not in df.columns:
            continue
        
        event_data = df[df[col] == 1].copy()
        
        if len(event_data) == 0:
            continue
        
        # 各日のランク計算
        top_counts = {}
        for digit in DIGIT_ORDER:
            top_counts[digit] = 0
        
        for date in event_data['date'].unique():
            date_data = event_data[event_data['date'] == date].copy()
            date_data = date_data.sort_values('avg_diff_coins', ascending=False)
            
            if rank_type == 'top1' and len(date_data) >= 1:
                top_digit = date_data.iloc[0]['last_digit']
                top_counts[top_digit] += 1
            elif rank_type == 'top2' and len(date_data) >= 2:
                top2_digit = date_data.iloc[1]['last_digit']
                top_counts[top2_digit] += 1
        
        pivot_data.append(pd.Series(top_counts, name=label))
    
    if len(pivot_data) == 0:
        print(f"No data for {title}")
        return
    
    pivot_df = pd.concat(pivot_data, axis=1).T
    
    # ヒートマップ描画
    fig, ax = plt.subplots(figsize=(14, max(8, len(pivot_data) * 0.5)))
    
    rank_label = 'TOP1' if rank_type == 'top1' else 'TOP2'
    cmap = 'YlOrRd' if rank_type == 'top1' else 'YlGnBu'
    
    sns.heatmap(
        pivot_df, 
        annot=True, 
        fmt='.0f', 
        cmap=cmap, 
        ax=ax,
        cbar_kws={'label': f'{rank_label} Count'}
    )
    
    ax.set_title(f'{title} - {rank_label} Count by Digit x Event', 
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Last Digit', fontsize=12)
    ax.set_ylabel('Event Type', fontsize=12)
    plt.tight_layout()
    plt.show()


# 実行
print("="*100)
print("【TOP1/TOP2取得回数ヒートマップ】")
print("="*100)

# データ読み込み(未読み込みの場合)
if 'df_all' not in globals():
    conn = sqlite3.connect(DB_PATH)
    df_all = pd.read_sql_query("SELECT * FROM last_digit_summary_all", conn)
    df_events = pd.read_sql_query("SELECT * FROM event_calendar", conn)
    df_all = df_all.merge(df_events, on='date', how='left')
    df_all['date_obj'] = pd.to_datetime(df_all['date'], format='%Y%m%d')
    df_all['weekday_num'] = df_all['date_obj'].dt.weekday
    df_all['last_digit'] = pd.Categorical(df_all['last_digit'], 
                                          categories=DIGIT_ORDER, 
                                          ordered=True)
    conn.close()

# TOP1ヒートマップ
print("\n【TOP1取得回数】")
plot_top_rank_heatmap(df_all, 'ALL', rank_type='top1')

# TOP2ヒートマップ
print("\n【TOP2取得回数】")
plot_top_rank_heatmap(df_all, 'ALL', rank_type='top2')

print("\n" + "="*100)
print("✅ ヒートマップ表示完了")
print("="*100)

In [ ]:
# ============================================================
# セル5-1: ヒートマップ表示（1回実行）
# ============================================================

def plot_heatmap(df, title):
    """ヒートマップのみ表示"""
    df = df.copy()
    df['is_saturday'] = (df['weekday_num'] == 5).astype(int)
    df['is_sunday'] = (df['weekday_num'] == 6).astype(int)
    
    # ヒートマップ用データ
    pivot_data = []
    for col, label in EVENT_DEFINITIONS.items():
        if col not in df.columns:
            continue
        event_data = df[df[col] == 1]
        if len(event_data) > 0:
            pivot_data.append(event_data.groupby('last_digit')['avg_diff_coins'].mean().rename(label))
    
    if len(pivot_data) == 0:
        print(f"No data for {title}")
        return
    
    pivot_df = pd.concat(pivot_data, axis=1).T
    
    # ヒートマップ描画
    fig, ax = plt.subplots(figsize=(14, max(8, len(pivot_data) * 0.5)))
    sns.heatmap(pivot_df, annot=True, fmt='.0f', cmap='RdYlGn', center=0, ax=ax, 
                cbar_kws={'label': 'Avg Diff Coins'})
    ax.set_title(f'{title} - Avg Diff by Digit x Event', fontsize=14, fontweight='bold')
    ax.set_xlabel('Last Digit', fontsize=12)
    ax.set_ylabel('Event Type', fontsize=12)
    plt.tight_layout()
    plt.show()

# 全モデル実行
data_dict = {'all': df_all, 'jug': df_jug, 'non_jug': df_non_jug}

for name, df in data_dict.items():
    if len(df) > 0:
        print(f"【{name.upper()}】")
        plot_heatmap(df, name.upper())
        print("\n" + "="*100 + "\n")

print("✅ ヒートマップ表示完了")

In [ ]:
# ============================================================
# セル5-2: LLM用JSONデータ出力（1回実行）
# ============================================================

def export_json_data(trend_data, title):
    """JSONデータ出力"""
    print(f"\n{'='*80}")
    print(f"【{title} - LLM用データ】")
    print(f"{'='*80}\n")
    print(json.dumps(trend_data, indent=2, ensure_ascii=False))

# 全モデル実行
print("="*100)
print("AI分析用テキストデータ出力開始")
print("="*100)

for name in ['all', 'jug', 'non_jug']:
    if name in trends and len(trends[name]) > 0:
        export_json_data(trends[name], name.upper())
        print("\n" + "="*100 + "\n")

print("✅ LLM用データ出力完了")

In [ ]:
# ============================================================
# セル6: LogReg学習（1回実行）
# ============================================================

def train_logreg_all_events(history_df, model_name, n_test=5):
    """全イベントタイプ一括学習"""
    results_all = {}
    models_all = {}
    
    for event_type in sorted(history_df['event_type'].unique()):
        event_history = history_df[history_df['event_type'] == event_type].copy()
        event_dates = sorted(event_history['date'].unique())
        
        if len(event_dates) < n_test + 3:
            continue
        
        test_dates = event_dates[-n_test:]
        results = []
        
        print(f"\n{'='*80}")
        print(f"【{event_type.upper()} - {model_name}】")
        print(f"{'='*80}\n")
        
        for test_date in test_dates:
            train_cutoff = event_dates[event_dates.index(test_date) - 1]
            X_train, y_train, _ = prepare_enhanced_features(history_df, event_type, train_cutoff, include_current=True)
            
            if X_train is None or len(X_train) == 0:
                continue
            
            # テストデータ: test_dateを除外して特徴量のみ取得
            test_data = event_history[event_history['date'] == test_date]
            X_test_base, _, _ = prepare_enhanced_features(history_df, event_type, test_date, include_current=False)
            
            if X_test_base is None:
                continue
            
            # test_date当日の末尾に対応する特徴量を取得
            X_test_list = []
            for digit in test_data['last_digit'].values:
                # この末尾の最新特徴量（test_date以前）
                digit_history = history_df[
                    (history_df['event_type'] == event_type) & 
                    (history_df['last_digit'] == digit) &
                    (history_df['date'] < test_date)
                ]
                if len(digit_history) > 0:
                    latest = digit_history.iloc[-1]
                    X_test_list.append([
                        latest['prev_1_diff'], latest['prev_2_diff'],
                        latest['prev_1_change'], latest['prev_2_change']
                    ])
                else:
                    X_test_list.append([0, 0, 0, 0])
            
            X_test = pd.DataFrame(X_test_list, columns=['prev_1_diff', 'prev_2_diff', 'prev_1_change', 'prev_2_change'])
            
            # 学習・予測
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train.fillna(0))
            X_test_scaled = scaler.transform(X_test.fillna(0))
            
            model = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000, random_state=42)
            model.fit(X_train_scaled, y_train)
            
            proba = model.predict_proba(X_test_scaled)[:, 1]
            predicted_top3 = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba), 
                                                    key=lambda x: x[1], reverse=True)[:3]]
            actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
            overlap = len(set(predicted_top3) & set(actual_top3))
            
            results.append({
                'date': test_date, 'predicted_top3': predicted_top3,
                'actual_top3': actual_top3, 'overlap': overlap, 'precision': overlap / 3
            })
        
        if len(results) > 0:
            df_results = pd.DataFrame(results)
            print(f"{'日付':<12} {'予測TOP3':<30} {'実際TOP3':<30} {'的中':<6} {'Precision':<10}")
            print("-" * 100)
            for _, row in df_results.iterrows():
                pred = ','.join([str(d) for d in row['predicted_top3']])
                actual = ','.join([str(d) for d in row['actual_top3']])
                print(f"{row['date']:<12} {pred:<30} {actual:<30} {row['overlap']:<6} {row['precision']:<10.3f}")
            print(f"\n平均 Precision@3: {df_results['precision'].mean():.3f}")
            
            results_all[event_type] = df_results
            
            # 最終モデル学習
            X_all, y_all, _ = prepare_enhanced_features(history_df, event_type, event_dates[-1])
            if X_all is not None:
                scaler_final = StandardScaler()
                X_all_scaled = scaler_final.fit_transform(X_all.fillna(0))
                model_final = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000, random_state=42)
                model_final.fit(X_all_scaled, y_all)
                models_all[event_type] = {'model': model_final, 'scaler': scaler_final}
    
    return results_all, models_all

# 3モデル学習
all_logreg_results = {}
all_logreg_models = {}

for name in ['all', 'jug', 'non_jug']:
    if name in histories and len(histories[name]) > 0:
        print(f"\n{'='*100}")
        print(f"{name.upper()} LogReg学習")
        print(f"{'='*100}")
        results, models = train_logreg_all_events(histories[name], name.upper(), n_test=5)
        all_logreg_results[name] = results
        all_logreg_models[name] = models

print("\n✅ LogReg学習完了")

In [ ]:
# ============================================================
# セル7: 次回イベント予測（1回実行）
# ============================================================

def predict_next_event(history_df, event_type, model_info):
    """次回イベント予測"""
    event_history = history_df[history_df['event_type'] == event_type].copy()
    if len(event_history) == 0:
        return None
    
    model, scaler = model_info['model'], model_info['scaler']
    latest_date = event_history['date'].max()
    
    X_all, _, _ = prepare_enhanced_features(history_df, event_type, latest_date)
    if X_all is None:
        return None
    
    latest_data = event_history[event_history['date'] == latest_date]
    X_latest = X_all.iloc[-len(latest_data):].fillna(0)
    
    proba = model.predict_proba(scaler.transform(X_latest))[:, 1]
    return pd.DataFrame({
        'digit': latest_data['last_digit'].values,
        'probability': proba
    }).sort_values('probability', ascending=False).reset_index(drop=True)

def display_all_predictions():
    """全予測表示"""
    for name in ['all', 'jug', 'non_jug']:
        if name not in all_logreg_models:
            continue
        
        for event_type, model_info in sorted(all_logreg_models[name].items()):
            pred = predict_next_event(histories[name], event_type, model_info)
            if pred is None:
                continue
            
            print(f"\n{'='*100}")
            print(f"【次回 {event_type.upper()} 予測 - {name.upper()}】")
            print(f"{'='*100}\n")
            print(f"{'順位':<6} {'末尾':<12} {'確率':>10}")
            print("-" * 30)
            for idx, row in pred.iterrows():
                print(f"{idx+1:<6} {row['digit']:<12} {row['probability']:>10.3f}")

display_all_predictions()

print("\n✅ 次回イベント予測完了")

In [ ]:
# ============================================================
# セル8: 特徴量重要度分析（1回実行）
# ============================================================

def analyze_feature_importance_all(model_dict, history_df, model_name):
    """全イベント特徴量重要度分析"""
    print(f"\n{'='*100}")
    print(f"【{model_name} - 特徴量重要度】")
    print(f"{'='*100}\n")
    
    for event, model_info in sorted(model_dict.items()):
        event_history = history_df[history_df['event_type'] == event]
        latest_date = event_history['date'].max()
        X_sample, _, _ = prepare_enhanced_features(history_df, event, latest_date)
        
        if X_sample is None:
            continue
        
        coefficients = model_info['model'].coef_[0]
        importance = pd.DataFrame({
            'feature': X_sample.columns,
            'coefficient': coefficients,
            'abs_coefficient': np.abs(coefficients)
        }).sort_values('abs_coefficient', ascending=False)
        
        print(f"\n{event.upper()}:")
        print(f"{'順位':<6} {'特徴量':<35} {'係数':>12} {'絶対値':>12}")
        print("-" * 70)
        for idx, (_, row) in enumerate(importance.head(15).iterrows(), 1):
            print(f"{idx:<6} {row['feature']:<35} {row['coefficient']:>12.4f} {row['abs_coefficient']:>12.4f}")

# 全モデル実行
for name in ['all', 'jug', 'non_jug']:
    if name in all_logreg_models and len(all_logreg_models[name]) > 0:
        analyze_feature_importance_all(all_logreg_models[name], histories[name], name.upper())

print("\n✅ 特徴量重要度分析完了")

In [ ]:
# ============================================================
# 特徴量エンジニアリング実験セル
# ============================================================

def create_enhanced_history_features(df):
    """拡張特徴量付き履歴作成（ランク追加）"""
    all_dates = sorted(df['date'].unique())
    
    print("  インデックス構築中...")
    event_digit_index = {}
    
    for date in all_dates:
        date_mask = df['date'] == date
        date_data = df[date_mask].iloc[0]
        
        active_events = []
        for col, label in EVENT_DEFINITIONS.items():
            if col in df.columns and date_data.get(col, 0) == 1:
                active_events.append(label.lower())
        
        for digit in df[date_mask]['last_digit'].unique():
            for event_name in active_events:
                key = (event_name, digit)
                if key not in event_digit_index:
                    event_digit_index[key] = []
                event_digit_index[key].append(date)
    
    # キャッシュ作成
    diff_cache = df.set_index(['date', 'last_digit'])['avg_diff_coins'].to_dict()
    games_cache = df.set_index(['date', 'last_digit'])['avg_games'].to_dict()
    
    # ランクキャッシュ（利用可能なら）
    rank_diff_cache = {}
    rank_games_cache = {}
    if 'last_digit_rank_diff' in df.columns:
        rank_diff_cache = df.set_index(['date', 'last_digit'])['last_digit_rank_diff'].to_dict()
    if 'last_digit_rank_games' in df.columns:
        rank_games_cache = df.set_index(['date', 'last_digit'])['last_digit_rank_games'].to_dict()
    
    # 日付×末尾のソート済みリスト
    date_digit_pairs = sorted(set((row['date'], row['last_digit']) for _, row in df.iterrows()))
    date_digit_index = {}
    for date, digit in date_digit_pairs:
        if digit not in date_digit_index:
            date_digit_index[digit] = []
        date_digit_index[digit].append(date)
    
    print("  拡張特徴量生成中...")
    history_features = []
    
    for (event_name, digit), dates in event_digit_index.items():
        for i, date in enumerate(dates):
            current_diff = diff_cache.get((date, digit))
            current_games = games_cache.get((date, digit))
            
            if current_diff is None or current_games is None:
                continue
            
            # イベント依存（過去同イベント）
            prev_diffs = [diff_cache.get((dates[i-j], digit)) for j in range(1, 4) if i >= j]
            prev_games = [games_cache.get((dates[i-j], digit)) for j in range(1, 4) if i >= j]
            prev_diffs = [v for v in prev_diffs if v is not None]
            prev_games = [v for v in prev_games if v is not None]
            
            # イベント非依存（直近7日）
            digit_history = date_digit_index.get(digit, [])
            current_idx = digit_history.index(date) if date in digit_history else -1
            
            recent_diffs = []
            recent_games = []
            recent_ranks_diff = []
            recent_ranks_games = []
            
            if current_idx > 0:
                for j in range(1, min(8, current_idx + 1)):
                    recent_date = digit_history[current_idx - j]
                    rd = diff_cache.get((recent_date, digit))
                    rg = games_cache.get((recent_date, digit))
                    rrd = rank_diff_cache.get((recent_date, digit))
                    rrg = rank_games_cache.get((recent_date, digit))
                    
                    if rd is not None:
                        recent_diffs.append(rd)
                    if rg is not None:
                        recent_games.append(rg)
                    if rrd is not None:
                        recent_ranks_diff.append(rrd)
                    if rrg is not None:
                        recent_ranks_games.append(rrg)
            
            features = {
                'date': date, 'last_digit': digit, 'event_type': event_name,
                'current_diff': current_diff,
                
                # イベント依存：基本
                'prev_1_diff': prev_diffs[0] if len(prev_diffs) > 0 else 0,
                'prev_1_games': prev_games[0] if len(prev_games) > 0 else 0,
                
                # 直近履歴：差枚・ゲーム数
                'recent_mean_diff': np.mean(recent_diffs) if len(recent_diffs) > 0 else 0,
                'recent_max_diff': np.max(recent_diffs) if len(recent_diffs) > 0 else 0,
                'recent_mean_games': np.mean(recent_games) if len(recent_games) > 0 else 0,
                'recent_max_games': np.max(recent_games) if len(recent_games) > 0 else 0,
                
                # 直近履歴：ランク
                'recent_mean_rank_diff': np.mean(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'recent_best_rank_diff': np.min(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'recent_mean_rank_games': np.mean(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                'recent_best_rank_games': np.min(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                
                # ランク安定性（標準偏差が小さい=安定上位）
                'rank_diff_stability': np.std(recent_ranks_diff) if len(recent_ranks_diff) > 1 else 0,
                'rank_games_stability': np.std(recent_ranks_games) if len(recent_ranks_games) > 1 else 0,
                
                # TOP3率（直近7日でランク3位以内の割合）
                'top3_rate_diff': sum(1 for r in recent_ranks_diff if r <= 3) / len(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'top3_rate_games': sum(1 for r in recent_ranks_games if r <= 3) / len(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                
                # データ充実度
                'data_count': len(recent_diffs),
            }
            
            history_features.append(features)
    
    return pd.DataFrame(history_features)

def prepare_features_flexible(history_df, event_type, cutoff_date, include_current=False, 
                               feature_set='basic'):
    """柔軟な特徴量選択"""
    if include_current:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] <= cutoff_date)
        ].copy()
    else:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] < cutoff_date)
        ].copy()
    
    if len(event_data) == 0:
        return None, None, None
    
    # 特徴量セット定義
    feature_sets = {
        'basic': ['prev_1_diff', 'prev_1_games'],
        
        'rank_focused': ['recent_mean_rank_diff', 'recent_best_rank_diff', 'top3_rate_diff',
                        'recent_mean_rank_games', 'top3_rate_games'],
        
        'rank_and_perf': ['recent_mean_diff', 'recent_max_games', 
                         'recent_mean_rank_diff', 'recent_best_rank_games', 'top3_rate_diff'],
        
        'all': ['prev_1_diff', 'prev_1_games',
               'recent_mean_diff', 'recent_max_diff', 'recent_mean_games', 'recent_max_games',
               'recent_mean_rank_diff', 'recent_best_rank_diff', 'recent_mean_rank_games', 'recent_best_rank_games',
               'rank_diff_stability', 'rank_games_stability', 'top3_rate_diff', 'top3_rate_games', 'data_count']
    }
    
    selected_features = [f for f in feature_sets[feature_set] if f in event_data.columns]
    X = event_data[selected_features].fillna(0)
    y = (event_data['current_diff'] > event_data['current_diff'].median()).astype(int)
    
    return X, y, event_data

def compare_feature_sets(history_df, model_name, n_test=5):
    """複数特徴量セットの比較"""
    feature_sets = ['basic', 'rank_focused', 'rank_and_perf', 'all']
    results_summary = []
    
    for feature_set in feature_sets:
        print(f"\n{'='*80}")
        print(f"【{feature_set.upper()} 特徴量セット - {model_name}】")
        print(f"{'='*80}")
        
        set_results = []
        
        for event_type in sorted(history_df['event_type'].unique())[:3]:  # 上位3イベントで検証
            event_history = history_df[history_df['event_type'] == event_type].copy()
            event_dates = sorted(event_history['date'].unique())
            
            if len(event_dates) < n_test + 3:
                continue
            
            test_dates = event_dates[-n_test:]
            precisions = []
            
            for test_date in test_dates:
                train_cutoff = event_dates[event_dates.index(test_date) - 1]
                X_train, y_train, _ = prepare_features_flexible(
                    history_df, event_type, train_cutoff, include_current=True, feature_set=feature_set
                )
                
                if X_train is None or len(X_train) == 0 or X_train.shape[1] == 0:
                    print(f"  スキップ: {event_type}/{test_date} - 特徴量なし")
                    continue
                
                test_data = event_history[event_history['date'] == test_date]
                X_test_list = []
                
                for digit in test_data['last_digit'].values:
                    digit_history = history_df[
                        (history_df['event_type'] == event_type) & 
                        (history_df['last_digit'] == digit) &
                        (history_df['date'] < test_date)
                    ]
                    if len(digit_history) > 0:
                        latest = digit_history.iloc[-1]
                        X_test_list.append([latest.get(f, 0) for f in X_train.columns])
                    else:
                        X_test_list.append([0] * len(X_train.columns))
                
                X_test = pd.DataFrame(X_test_list, columns=X_train.columns)
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
                
                model = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000, random_state=42)
                model.fit(X_train_scaled, y_train)
                
                proba = model.predict_proba(X_test_scaled)[:, 1]
                predicted_top3 = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba), 
                                                        key=lambda x: x[1], reverse=True)[:3]]
                actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
                overlap = len(set(predicted_top3) & set(actual_top3))
                precisions.append(overlap / 3)
            
            if len(precisions) > 0:
                set_results.append({
                    'event': event_type,
                    'precision': np.mean(precisions),
                    'n_features': len(X_train.columns)
                })
        
        if len(set_results) > 0:
            avg_precision = np.mean([r['precision'] for r in set_results])
            avg_n_features = np.mean([r['n_features'] for r in set_results])
            
            print(f"\n平均 Precision@3: {avg_precision:.3f}")
            print(f"平均特徴量数: {avg_n_features:.0f}")
            
            results_summary.append({
                'feature_set': feature_set,
                'model': model_name,
                'avg_precision': avg_precision,
                'avg_n_features': avg_n_features
            })
    
    return pd.DataFrame(results_summary)

# 実行
print("拡張特徴量履歴作成中...")
enhanced_histories = {}
for name, df in [('all', df_all), ('jug', df_jug), ('non_jug', df_non_jug)]:
    if len(df) > 0:
        print(f"\n【{name.upper()}】")
        enhanced_histories[name] = create_enhanced_history_features(df)

print("\n" + "="*100)
print("特徴量セット比較実験")
print("="*100)

comparison_results = []
for name in ['all', 'jug', 'non_jug']:
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = compare_feature_sets(enhanced_histories[name], name.upper(), n_test=5)
        comparison_results.append(result)

# サマリー表示
if len(comparison_results) > 0:
    final_summary = pd.concat(comparison_results, ignore_index=True)
    print("\n" + "="*100)
    print("【総合結果サマリー】")
    print("="*100)
    print(final_summary.to_string(index=False))
    
    print("\n【ベスト特徴量セット】")
    best = final_summary.loc[final_summary.groupby('model')['avg_precision'].idxmax()]
    print(best[['model', 'feature_set', 'avg_precision', 'avg_n_features']].to_string(index=False))

print("\n✅ 特徴量エンジニアリング実験完了")

In [ ]:
# ============================================================
# XGBoost/RandomForest特徴量エンジニアリング実験
# ============================================================

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

def compare_tree_models(history_df, model_name, n_test=5):
    """XGBoost/RandomForest/LogRegの3モデル比較"""
    feature_sets = ['basic', 'rank_focused', 'rank_and_perf', 'all']
    results_summary = []
    
    for feature_set in feature_sets:
        print(f"\n{'='*80}")
        print(f"【{feature_set.upper()} - {model_name}】")
        print(f"{'='*80}")
        
        for event_type in sorted(history_df['event_type'].unique())[:3]:
            event_history = history_df[history_df['event_type'] == event_type].copy()
            event_dates = sorted(event_history['date'].unique())
            
            if len(event_dates) < n_test + 3:
                continue
            
            test_dates = event_dates[-n_test:]
            
            # 3モデルの結果格納
            model_results = {'LogReg': [], 'XGB': [], 'RF': []}
            
            for test_date in test_dates:
                train_cutoff = event_dates[event_dates.index(test_date) - 1]
                X_train, y_train, _ = prepare_features_flexible(
                    history_df, event_type, train_cutoff, include_current=True, feature_set=feature_set
                )
                
                if X_train is None or len(X_train) == 0 or X_train.shape[1] == 0:
                    continue
                
                test_data = event_history[event_history['date'] == test_date]
                X_test_list = []
                
                for digit in test_data['last_digit'].values:
                    digit_history = history_df[
                        (history_df['event_type'] == event_type) & 
                        (history_df['last_digit'] == digit) &
                        (history_df['date'] < test_date)
                    ]
                    if len(digit_history) > 0:
                        latest = digit_history.iloc[-1]
                        X_test_list.append([latest.get(f, 0) for f in X_train.columns])
                    else:
                        X_test_list.append([0] * len(X_train.columns))
                
                X_test = pd.DataFrame(X_test_list, columns=X_train.columns)
                
                # 正解データ
                actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
                
                # LogisticRegression
                scaler_lr = StandardScaler()
                X_train_lr = scaler_lr.fit_transform(X_train)
                X_test_lr = scaler_lr.transform(X_test)
                
                lr = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000, random_state=42)
                lr.fit(X_train_lr, y_train)
                proba_lr = lr.predict_proba(X_test_lr)[:, 1]
                pred_lr = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba_lr), 
                                                key=lambda x: x[1], reverse=True)[:3]]
                model_results['LogReg'].append(len(set(pred_lr) & set(actual_top3)) / 3)
                
                # XGBoost
                xgb = XGBClassifier(
                    n_estimators=100, max_depth=5, learning_rate=0.1,
                    subsample=0.8, colsample_bytree=0.8,
                    random_state=42, eval_metric='logloss'
                )
                xgb.fit(X_train, y_train)
                proba_xgb = xgb.predict_proba(X_test)[:, 1]
                pred_xgb = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba_xgb), 
                                                 key=lambda x: x[1], reverse=True)[:3]]
                model_results['XGB'].append(len(set(pred_xgb) & set(actual_top3)) / 3)
                
                # RandomForest
                rf = RandomForestClassifier(
                    n_estimators=100, max_depth=10, min_samples_split=5,
                    random_state=42, class_weight='balanced', n_jobs=-1
                )
                rf.fit(X_train, y_train)
                proba_rf = rf.predict_proba(X_test)[:, 1]
                pred_rf = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba_rf), 
                                                key=lambda x: x[1], reverse=True)[:3]]
                model_results['RF'].append(len(set(pred_rf) & set(actual_top3)) / 3)
            
            if len(model_results['LogReg']) > 0:
                avg_lr = np.mean(model_results['LogReg'])
                avg_xgb = np.mean(model_results['XGB'])
                avg_rf = np.mean(model_results['RF'])
                
                print(f"\n{event_type}:")
                print(f"  LogReg: {avg_lr:.3f}")
                print(f"  XGBoost: {avg_xgb:.3f}")
                print(f"  RandomForest: {avg_rf:.3f}")
                
                results_summary.append({
                    'feature_set': feature_set,
                    'model_type': model_name,
                    'event': event_type,
                    'LogReg': avg_lr,
                    'XGB': avg_xgb,
                    'RF': avg_rf,
                    'n_features': len(X_train.columns)
                })
    
    return pd.DataFrame(results_summary)

# 実行
print("="*100)
print("XGBoost/RandomForest 特徴量エンジニアリング実験")
print("="*100)

tree_comparison_results = []
for name in ['all', 'jug', 'non_jug']:
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = compare_tree_models(enhanced_histories[name], name.upper(), n_test=5)
        tree_comparison_results.append(result)

# サマリー表示
if len(tree_comparison_results) > 0:
    final_tree_summary = pd.concat(tree_comparison_results, ignore_index=True)
    
    print("\n" + "="*100)
    print("【総合結果サマリー】")
    print("="*100)
    print(final_tree_summary.to_string(index=False))
    
    # モデル別平均
    print("\n【モデル別平均Precision@3】")
    for model_type in final_tree_summary['model_type'].unique():
        subset = final_tree_summary[final_tree_summary['model_type'] == model_type]
        print(f"\n{model_type}:")
        print(f"  LogReg: {subset['LogReg'].mean():.3f}")
        print(f"  XGBoost: {subset['XGB'].mean():.3f}")
        print(f"  RandomForest: {subset['RF'].mean():.3f}")
    
    # ベスト組み合わせ
    print("\n【ベストモデル×特徴量セット】")
    for model_type in final_tree_summary['model_type'].unique():
        subset = final_tree_summary[final_tree_summary['model_type'] == model_type]
        
        best_lr = subset.loc[subset['LogReg'].idxmax()]
        best_xgb = subset.loc[subset['XGB'].idxmax()]
        best_rf = subset.loc[subset['RF'].idxmax()]
        
        print(f"\n{model_type}:")
        print(f"  LogReg best: {best_lr['feature_set']} ({best_lr['LogReg']:.3f})")
        print(f"  XGBoost best: {best_xgb['feature_set']} ({best_xgb['XGB']:.3f})")
        print(f"  RandomForest best: {best_rf['feature_set']} ({best_rf['RF']:.3f})")

print("\n✅ ツリーモデル比較実験完了")

In [ ]:
# ============================================================
# TPOT AutoML実験（安定化・Daskローカル対応版）
# ============================================================

# !pip install tpot dask distributed

import numpy as np
import pandas as pd
from tpot import TPOTClassifier

# Daskローカルクライアント起動（ワーカー不在エラー回避）
try:
    from dask.distributed import Client
    client = Client(processes=False, n_workers=2, threads_per_worker=2)
    print(client)
except Exception as e:
    client = None
    print(f"Dask client not started: {e}")

def run_tpot_automl(history_df, model_name, event_types=None, n_test=2):
    """TPOT AutoML実行（互換・ガード強化版）"""
    if event_types is None:
        if 'event_type' not in history_df.columns:
            print(f"[{model_name}] 'event_type' 列が見つかりません。処理をスキップします。")
            return pd.DataFrame()
        event_types = sorted(history_df['event_type'].unique())[:2]

    results = []

    for ev in event_types:
        df_ev = history_df[history_df['event_type'] == ev].copy()
        if len(df_ev) < 40:
            print(f"[{model_name}:{ev}] skip: samples too small ({len(df_ev)})")
            continue

        # 時系列順（date, last_digitがあれば）で並べる
        sort_cols = [c for c in ['date', 'last_digit'] if c in df_ev.columns]
        if sort_cols:
            df_ev = df_ev.sort_values(sort_cols).reset_index(drop=True)

        # 特徴量候補
        available_features = [
            c for c in df_ev.columns
            if c.startswith(('prev_', 'digit_num', 'is_', 'weekday_num', 'day_of_month'))
        ]
        if not available_features:
            print(f"[{model_name}:{ev}] skip: no available features")
            continue

        # テスト期間（末尾 n_test 日）
        if 'date' not in df_ev.columns:
            print(f"[{model_name}:{ev}] skip: 'date' 列なし")
            continue

        unique_dates = sorted(df_ev['date'].unique())
        if len(unique_dates) <= n_test:
            print(f"[{model_name}:{ev}] skip: not enough dates for test split")
            continue

        test_dates = set(unique_dates[-n_test:])
        train_data = df_ev[~df_ev['date'].isin(test_dates)].copy()
        test_data  = df_ev[df_ev['date'].isin(test_dates)].copy()

        # 前処理: NaN/inf -> 0
        def clean(X):
            X = X.replace([np.inf, -np.inf], 0)
            return X.fillna(0)

        X_train = clean(train_data[available_features])
        X_test  = clean(test_data[available_features])

        # ラベル: median超で1（既存仕様踏襲）
        if 'current_diff' not in train_data.columns or 'current_diff' not in test_data.columns:
            print(f"[{model_name}:{ev}] skip: 'current_diff' 列なし")
            continue

        y_train = (train_data['current_diff'] > train_data['current_diff'].median()).astype(int).values

        # 単一クラスをスキップ
        if len(np.unique(y_train)) < 2:
            print(f"[{model_name}:{ev}] skip: y_train single class (counts={np.bincount(y_train) if y_train.size else []})")
            continue

        # TPOT 初期化（バージョン互換のため最小引数のみ、Daskクライアントを渡す）
        def build_tpot(with_client=True):
            common = dict(
                generations=2,          # 軽め
                population_size=10,     # 軽め
                cv=3,                   # 互換性のため整数指定
                n_jobs=-1,
                verbose=2,              # 旧: verbosity
                random_state=42,
                max_time_mins=10,
                max_eval_time_mins=2
            )
            if with_client and client is not None:
                return TPOTClassifier(client=client, **common)
            else:
                return TPOTClassifier(**common)

        try:
            try:
                tpot = build_tpot(with_client=True)
                tpot.fit(X_train, y_train)
            except TypeError:
                # 一部版で client 引数が未対応な場合のフォールバック
                tpot = build_tpot(with_client=False)
                tpot.fit(X_train, y_train)
        except Exception as e:
            print(f"[{model_name}:{ev}] TPOT fit error: {e}; skipping.")
            continue

        # 予測（predict_proba未対応の器に備えフォールバック）
        try:
            proba = tpot.predict_proba(X_test)[:, 1]
        except Exception:
            proba = tpot.predict(X_test).astype(float)

        if 'last_digit' not in test_data.columns:
            print(f"[{model_name}:{ev}] warning: 'last_digit' 列なし（Top3計算をスキップ）")
            continue

        # Top3（確率上位3の末尾）
        predicted_top3 = [
            d for d, _ in sorted(
                zip(test_data['last_digit'].values, proba),
                key=lambda x: x[1],
                reverse=True
            )[:3]
        ]
        actual_top3 = [
            d for d, _ in sorted(
                zip(test_data['last_digit'].values, test_data['current_diff'].values),
                key=lambda x: x[1],
                reverse=True
            )[:3]
        ]
        overlap = len(set(predicted_top3) & set(actual_top3))

        results.append({
            'event': ev,
            'precision': overlap / 3.0,
            'predicted': predicted_top3,
            'actual': actual_top3
        })

        # 参考出力
        try:
            print(f"ベストパイプライン: {type(tpot.fitted_pipeline_).__name__}")
        except Exception:
            pass
        print(f"Precision@3: {overlap/3:.3f}")

    return pd.DataFrame(results)


# 実行
print("="*100)
print("TPOT AutoML実験")
print("="*100)

tpot_results = []
for name in ['all']:  # 時間削減のため1モデルのみ
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = run_tpot_automl(enhanced_histories[name], name.upper(), n_test=2)
        if result is not None and not result.empty:
            tpot_results.append(result)

# サマリー（安全化）
if len(tpot_results) > 0 and any((len(df) > 0 for df in tpot_results)):
    final_tpot = pd.concat(tpot_results, ignore_index=True)
    if 'precision' in final_tpot.columns and not final_tpot.empty:
        print(f"\n平均Precision@3: {final_tpot['precision'].mean():.3f}")
        print("\n詳細:")
        cols = [c for c in ['event', 'precision', 'predicted', 'actual'] if c in final_tpot.columns]
        print(final_tpot[cols].to_string(index=False))
    else:
        print("\n有効な結果がありません（すべてスキップ/失敗）。時間設定や分割条件を見直してください。")
else:
    print("\n結果が空です。")

# Dask後片付け（任意）
try:
    if client is not None:
        client.close()
except:
    pass

print("\n✅ TPOT AutoML完了")

In [ ]:
# ============================================================
# 特徴量エンジニアリング実験セル
# ============================================================

def create_enhanced_history_features(df):
    """拡張特徴量付き履歴作成（イベント×末尾マッチング追加）"""
    all_dates = sorted(df['date'].unique())
    
    print("  インデックス構築中...")
    event_digit_index = {}
    
    for date in all_dates:
        date_mask = df['date'] == date
        date_data = df[date_mask].iloc[0]
        
        active_events = []
        for col, label in EVENT_DEFINITIONS.items():
            if col in df.columns and date_data.get(col, 0) == 1:
                active_events.append(label.lower())
        
        for digit in df[date_mask]['last_digit'].unique():
            for event_name in active_events:
                key = (event_name, digit)
                if key not in event_digit_index:
                    event_digit_index[key] = []
                event_digit_index[key].append(date)
    
    # キャッシュ作成
    diff_cache = df.set_index(['date', 'last_digit'])['avg_diff_coins'].to_dict()
    games_cache = df.set_index(['date', 'last_digit'])['avg_games'].to_dict()
    
    # ランクキャッシュ
    rank_diff_cache = {}
    rank_games_cache = {}
    if 'last_digit_rank_diff' in df.columns:
        rank_diff_cache = df.set_index(['date', 'last_digit'])['last_digit_rank_diff'].to_dict()
    if 'last_digit_rank_games' in df.columns:
        rank_games_cache = df.set_index(['date', 'last_digit'])['last_digit_rank_games'].to_dict()
    
    # 日付×末尾のソート済みリスト
    date_digit_pairs = sorted(set((row['date'], row['last_digit']) for _, row in df.iterrows()))
    date_digit_index = {}
    for date, digit in date_digit_pairs:
        if digit not in date_digit_index:
            date_digit_index[digit] = []
        date_digit_index[digit].append(date)
    
    print("  拡張特徴量生成中...")
    history_features = []
    
    for (event_name, digit), dates in event_digit_index.items():
        # イベント×末尾マッチング判定
        is_matching = False
        if event_name in ['1day', '2day', '3day', '4day', '5day', '6day', '7day', '8day', '9day', '0day']:
            event_digit = event_name[0]  # '7day' → '7'
            is_matching = (str(digit) == event_digit)
        
        # このイベント×末尾の過去実績集計
        matching_history_diffs = []
        matching_history_ranks = []
        
        for date in dates:
            d = diff_cache.get((date, digit))
            r = rank_diff_cache.get((date, digit))
            if d is not None:
                matching_history_diffs.append(d)
            if r is not None:
                matching_history_ranks.append(r)
        
        for i, date in enumerate(dates):
            current_diff = diff_cache.get((date, digit))
            current_games = games_cache.get((date, digit))
            
            if current_diff is None or current_games is None:
                continue
            
            # イベント依存（過去同イベント）
            prev_diffs = [diff_cache.get((dates[i-j], digit)) for j in range(1, 4) if i >= j]
            prev_games = [games_cache.get((dates[i-j], digit)) for j in range(1, 4) if i >= j]
            prev_diffs = [v for v in prev_diffs if v is not None]
            prev_games = [v for v in prev_games if v is not None]
            
            # イベント非依存（直近7日）
            digit_history = date_digit_index.get(digit, [])
            current_idx = digit_history.index(date) if date in digit_history else -1
            
            recent_diffs = []
            recent_games = []
            recent_ranks_diff = []
            recent_ranks_games = []
            
            if current_idx > 0:
                for j in range(1, min(8, current_idx + 1)):
                    recent_date = digit_history[current_idx - j]
                    rd = diff_cache.get((recent_date, digit))
                    rg = games_cache.get((recent_date, digit))
                    rrd = rank_diff_cache.get((recent_date, digit))
                    rrg = rank_games_cache.get((recent_date, digit))
                    
                    if rd is not None:
                        recent_diffs.append(rd)
                    if rg is not None:
                        recent_games.append(rg)
                    if rrd is not None:
                        recent_ranks_diff.append(rrd)
                    if rrg is not None:
                        recent_ranks_games.append(rrg)
            
            # マッチング特徴量（過去のこのイベント×末尾の実績、当日除外）
            past_matching_diffs = matching_history_diffs[:i] if i > 0 else []
            past_matching_ranks = matching_history_ranks[:i] if i > 0 else []
            
            features = {
                'date': date, 'last_digit': digit, 'event_type': event_name,
                'current_diff': current_diff,
                
                # イベント依存：基本
                'prev_1_diff': prev_diffs[0] if len(prev_diffs) > 0 else 0,
                'prev_1_games': prev_games[0] if len(prev_games) > 0 else 0,
                
                # 直近履歴：差枚・ゲーム数
                'recent_mean_diff': np.mean(recent_diffs) if len(recent_diffs) > 0 else 0,
                'recent_max_diff': np.max(recent_diffs) if len(recent_diffs) > 0 else 0,
                'recent_mean_games': np.mean(recent_games) if len(recent_games) > 0 else 0,
                'recent_max_games': np.max(recent_games) if len(recent_games) > 0 else 0,
                
                # 直近履歴：ランク
                'recent_mean_rank_diff': np.mean(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'recent_best_rank_diff': np.min(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'recent_mean_rank_games': np.mean(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                'recent_best_rank_games': np.min(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                
                # ランク安定性
                'rank_diff_stability': np.std(recent_ranks_diff) if len(recent_ranks_diff) > 1 else 0,
                'rank_games_stability': np.std(recent_ranks_games) if len(recent_ranks_games) > 1 else 0,
                
                # TOP3率
                'top3_rate_diff': sum(1 for r in recent_ranks_diff if r <= 3) / len(recent_ranks_diff) if len(recent_ranks_diff) > 0 else 0,
                'top3_rate_games': sum(1 for r in recent_ranks_games if r <= 3) / len(recent_ranks_games) if len(recent_ranks_games) > 0 else 0,
                
                # データ充実度
                'data_count': len(recent_diffs),
                
                # ★新規：イベント×末尾マッチング特徴量
                'is_matching_digit': 1 if is_matching else 0,
                'matching_history_mean': np.mean(past_matching_diffs) if len(past_matching_diffs) > 0 else 0,
                'matching_history_max': np.max(past_matching_diffs) if len(past_matching_diffs) > 0 else 0,
                'matching_history_count': len(past_matching_diffs),
                'matching_rank_mean': np.mean(past_matching_ranks) if len(past_matching_ranks) > 0 else 0,
                'matching_top3_rate': sum(1 for r in past_matching_ranks if r <= 3) / len(past_matching_ranks) if len(past_matching_ranks) > 0 else 0,
            }
            
            history_features.append(features)
    
    return pd.DataFrame(history_features)

def prepare_features_flexible(history_df, event_type, cutoff_date, include_current=False, 
                               feature_set='basic'):
    """柔軟な特徴量選択"""
    if include_current:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] <= cutoff_date)
        ].copy()
    else:
        event_data = history_df[
            (history_df['event_type'] == event_type) & 
            (history_df['date'] < cutoff_date)
        ].copy()
    
    if len(event_data) == 0:
        return None, None, None
    
    # 特徴量セット定義
    feature_sets = {
        'basic': ['prev_1_diff', 'prev_1_games'],
        
        'rank_focused': ['recent_mean_rank_diff', 'recent_best_rank_diff', 'top3_rate_diff',
                        'recent_mean_rank_games', 'top3_rate_games'],
        
        'rank_and_perf': ['recent_mean_diff', 'recent_max_games', 
                         'recent_mean_rank_diff', 'recent_best_rank_games', 'top3_rate_diff'],
        
        'all': ['prev_1_diff', 'prev_1_games',
               'recent_mean_diff', 'recent_max_diff', 'recent_mean_games', 'recent_max_games',
               'recent_mean_rank_diff', 'recent_best_rank_diff', 'recent_mean_rank_games', 'recent_best_rank_games',
               'rank_diff_stability', 'rank_games_stability', 'top3_rate_diff', 'top3_rate_games', 'data_count']
    }
    
    selected_features = [f for f in feature_sets[feature_set] if f in event_data.columns]
    X = event_data[selected_features].fillna(0)
    y = (event_data['current_diff'] > event_data['current_diff'].median()).astype(int)
    
    return X, y, event_data

def compare_feature_sets(history_df, model_name, n_test=5):
    """複数特徴量セットの比較"""
    feature_sets = ['basic', 'rank_focused', 'rank_and_perf', 'all']
    results_summary = []
    
    for feature_set in feature_sets:
        print(f"\n{'='*80}")
        print(f"【{feature_set.upper()} 特徴量セット - {model_name}】")
        print(f"{'='*80}")
        
        set_results = []
        
        for event_type in sorted(history_df['event_type'].unique())[:3]:  # 上位3イベントで検証
            event_history = history_df[history_df['event_type'] == event_type].copy()
            event_dates = sorted(event_history['date'].unique())
            
            if len(event_dates) < n_test + 3:
                continue
            
            test_dates = event_dates[-n_test:]
            precisions = []
            
            for test_date in test_dates:
                train_cutoff = event_dates[event_dates.index(test_date) - 1]
                X_train, y_train, _ = prepare_features_flexible(
                    history_df, event_type, train_cutoff, include_current=True, feature_set=feature_set
                )
                
                if X_train is None or len(X_train) == 0 or X_train.shape[1] == 0:
                    print(f"  スキップ: {event_type}/{test_date} - 特徴量なし")
                    continue
                
                test_data = event_history[event_history['date'] == test_date]
                X_test_list = []
                
                for digit in test_data['last_digit'].values:
                    digit_history = history_df[
                        (history_df['event_type'] == event_type) & 
                        (history_df['last_digit'] == digit) &
                        (history_df['date'] < test_date)
                    ]
                    if len(digit_history) > 0:
                        latest = digit_history.iloc[-1]
                        X_test_list.append([latest.get(f, 0) for f in X_train.columns])
                    else:
                        X_test_list.append([0] * len(X_train.columns))
                
                X_test = pd.DataFrame(X_test_list, columns=X_train.columns)
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
                
                model = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000, random_state=42)
                model.fit(X_train_scaled, y_train)
                
                proba = model.predict_proba(X_test_scaled)[:, 1]
                predicted_top3 = [d for d, _ in sorted(zip(test_data['last_digit'].values, proba), 
                                                        key=lambda x: x[1], reverse=True)[:3]]
                actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
                overlap = len(set(predicted_top3) & set(actual_top3))
                precisions.append(overlap / 3)
            
            if len(precisions) > 0:
                set_results.append({
                    'event': event_type,
                    'precision': np.mean(precisions),
                    'n_features': len(X_train.columns)
                })
        
        if len(set_results) > 0:
            avg_precision = np.mean([r['precision'] for r in set_results])
            avg_n_features = np.mean([r['n_features'] for r in set_results])
            
            print(f"\n平均 Precision@3: {avg_precision:.3f}")
            print(f"平均特徴量数: {avg_n_features:.0f}")
            
            results_summary.append({
                'feature_set': feature_set,
                'model': model_name,
                'avg_precision': avg_precision,
                'avg_n_features': avg_n_features
            })
    
    return pd.DataFrame(results_summary)

# 実行
print("拡張特徴量履歴作成中...")
enhanced_histories = {}
for name, df in [('all', df_all), ('jug', df_jug), ('non_jug', df_non_jug)]:
    if len(df) > 0:
        print(f"\n【{name.upper()}】")
        enhanced_histories[name] = create_enhanced_history_features(df)

print("\n" + "="*100)
print("特徴量セット比較実験")
print("="*100)

comparison_results = []
for name in ['all', 'jug', 'non_jug']:
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = compare_feature_sets(enhanced_histories[name], name.upper(), n_test=5)
        comparison_results.append(result)

# サマリー表示
if len(comparison_results) > 0:
    final_summary = pd.concat(comparison_results, ignore_index=True)
    print("\n" + "="*100)
    print("【総合結果サマリー】")
    print("="*100)
    print(final_summary.to_string(index=False))
    
    print("\n【ベスト特徴量セット】")
    best = final_summary.loc[final_summary.groupby('model')['avg_precision'].idxmax()]
    print(best[['model', 'feature_set', 'avg_precision', 'avg_n_features']].to_string(index=False))

print("\n✅ 特徴量エンジニアリング実験完了")

In [ ]:
# ============================================================
# FLAML AutoML（TOP3ラベル版）
# ============================================================

# pip install flaml


from flaml import AutoML

def run_flaml_top3_label(history_df, model_name, event_types=None, n_test=3, time_budget=180):
    """FLAML TOP3ラベル版（評価指標と訓練目的を一致）"""
    
    if event_types is None:
        # データ量の多い上位3イベント
        event_types = history_df['event_type'].value_counts().head(3).index.tolist()
    
    results = []
    
    for event_type in event_types:
        print(f"\n{'='*80}")
        print(f"【{event_type.upper()} - {model_name}】")
        print(f"{'='*80}\n")
        
        event_data = history_df[history_df['event_type'] == event_type].copy()
        event_dates = sorted(event_data['date'].unique())
        
        if len(event_dates) < n_test + 5:
            print(f"データ不足: {len(event_dates)} days")
            continue
        
        # ウォークフォワード検証
        test_dates = event_dates[-n_test:]
        
        for i, test_date in enumerate(test_dates):
            print(f"\nTest {i+1}/{n_test}: {test_date}")
            
            train_data = event_data[event_data['date'] < test_date].copy()
            test_data = event_data[event_data['date'] == test_date].copy()
            
            # TOP3ラベル作成（訓練データ）
            train_data['rank'] = train_data.groupby('date')['current_diff'].rank(ascending=False, method='first')
            train_data['label_top3'] = (train_data['rank'] <= 3).astype(int)
            
            # 特徴量
            exclude_cols = ['date', 'last_digit', 'event_type', 'current_diff', 'rank', 'label_top3']
            feature_cols = [c for c in train_data.columns if c not in exclude_cols]
            
            X_train = train_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
            y_train = train_data['label_top3'].values
            X_test = test_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
            
            if X_train.shape[1] == 0:
                print("  特徴量なし")
                continue
            
            # クラス分布確認
            pos_ratio = y_train.mean()
            print(f"  訓練: {len(X_train)} samples × {X_train.shape[1]} features")
            print(f"  TOP3比率: {pos_ratio:.3f} (期待値: 0.273)")
            
            # FLAML実行
            automl = AutoML()
            
            try:
                automl.fit(
                    X_train=X_train,
                    y_train=y_train,
                    task="classification",
                    time_budget=time_budget,
                    metric="ap",  # Average Precision（不均衡データ向け）
                    estimator_list=['lgbm', 'xgboost', 'rf', 'extra_tree'],
                    n_splits=3,
                    verbose=0,
                    seed=42,
                    early_stop=True
                )
                
                # 予測確率
                proba = automl.predict_proba(X_test)[:, 1]
                
                # Precision@3
                predicted_top3 = [
                    d for d, _ in sorted(
                        zip(test_data['last_digit'].values, proba),
                        key=lambda x: x[1],
                        reverse=True
                    )[:3]
                ]
                actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
                overlap = len(set(predicted_top3) & set(actual_top3))
                precision = overlap / 3
                
                print(f"  モデル: {automl.best_estimator}")
                print(f"  Precision@3: {precision:.3f}")
                print(f"  予測: {predicted_top3} / 実際: {actual_top3}")
                
                # 予測確率分布
                print(f"  予測確率範囲: {proba.min():.3f} - {proba.max():.3f}")
                print(f"  TOP3平均確率: {np.mean([proba[list(test_data['last_digit'].values).index(d)] for d in predicted_top3 if d in test_data['last_digit'].values]):.3f}")
                
                # 特徴量重要度（上位10）
                try:
                    if hasattr(automl.model.estimator, 'feature_importances_'):
                        imp = automl.model.estimator.feature_importances_
                        top_features = pd.Series(imp, index=feature_cols).nlargest(10)
                        print(f"  重要特徴量TOP10:")
                        for feat, val in top_features.items():
                            print(f"    {feat}: {val:.4f}")
                except:
                    pass
                
                results.append({
                    'model_name': model_name,
                    'event': event_type,
                    'test_date': test_date,
                    'precision': precision,
                    'best_model': automl.best_estimator,
                    'n_features': X_train.shape[1],
                    'pos_ratio': pos_ratio,
                    'predicted': predicted_top3,
                    'actual': actual_top3
                })
                
            except Exception as e:
                print(f"  エラー: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    return pd.DataFrame(results)

# 実行
print("="*100)
print("FLAML AutoML（TOP3ラベル版）")
print("="*100)

flaml_top3_results = []
for name in ['all', 'jug']:  # 2モデル
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = run_flaml_top3_label(
            enhanced_histories[name], 
            name.upper(), 
            event_types=None,
            n_test=3,
            time_budget=180
        )
        if result is not None and len(result) > 0:
            flaml_top3_results.append(result)

# サマリー
if flaml_top3_results:
    final_flaml = pd.concat(flaml_top3_results, ignore_index=True)
    
    print("\n" + "="*100)
    print("【FLAML TOP3ラベル結果サマリー】")
    print("="*100)
    
    # モデル別平均
    for model_name in final_flaml['model_name'].unique():
        subset = final_flaml[final_flaml['model_name'] == model_name]
        print(f"\n{model_name}:")
        print(f"  平均Precision@3: {subset['precision'].mean():.3f}")
        print(f"  最高Precision@3: {subset['precision'].max():.3f}")
        print(f"  使用モデル: {subset['best_model'].value_counts().to_dict()}")
    
    # 詳細結果
    print("\n【詳細結果】")
    display_cols = ['model_name', 'event', 'precision', 'best_model']
    print(final_flaml[display_cols].to_string(index=False))
    
    # ベスト結果
    best_idx = final_flaml['precision'].idxmax()
    best = final_flaml.loc[best_idx]
    print(f"\n【ベスト結果】")
    print(f"モデル: {best['model_name']} / イベント: {best['event']}")
    print(f"Precision@3: {best['precision']:.3f}")
    print(f"使用モデル: {best['best_model']} ({int(best['n_features'])} features)")
    print(f"予測: {best['predicted']} / 実際: {best['actual']}")

print("\n✅ FLAML TOP3ラベル版完了")

In [ ]:
# ============================================================
# ランク予測（回帰→ランク変換）
# ============================================================

from flaml import AutoML
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_rank_regression(history_df, model_name, event_types=None, n_test=3, time_budget=180):
    """回帰でランク予測 → TOP3 Precision評価"""
    
    if event_types is None:
        event_types = history_df['event_type'].value_counts().head(3).index.tolist()
    
    results = []
    
    for event_type in event_types:
        print(f"\n{'='*80}")
        print(f"【{event_type.upper()} - {model_name} (ランク回帰)】")
        print(f"{'='*80}\n")
        
        event_data = history_df[history_df['event_type'] == event_type].copy()
        event_dates = sorted(event_data['date'].unique())
        
        if len(event_dates) < n_test + 5:
            print(f"データ不足: {len(event_dates)} days")
            continue
        
        test_dates = event_dates[-n_test:]
        
        for i, test_date in enumerate(test_dates):
            print(f"\nTest {i+1}/{n_test}: {test_date}")
            
            train_data = event_data[event_data['date'] < test_date].copy()
            test_data = event_data[event_data['date'] == test_date].copy()
            
            # ランクラベル作成（1位=1, 11位=11）
            train_data['rank'] = train_data.groupby('date')['current_diff'].rank(
                ascending=False, method='first'
            ).astype(int)
            
            # 特徴量
            exclude_cols = ['date', 'last_digit', 'event_type', 'current_diff', 'rank']
            feature_cols = [c for c in train_data.columns if c not in exclude_cols]
            
            X_train = train_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
            y_train = train_data['rank'].values  # 回帰ターゲット
            X_test = test_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
            
            if X_train.shape[1] == 0:
                print("  特徴量なし")
                continue
            
            print(f"  訓練: {len(X_train)} samples × {X_train.shape[1]} features")
            print(f"  ランク範囲: {y_train.min()}-{y_train.max()}")
            
            # FLAML実行（回帰）
            automl = AutoML()
            
            try:
                automl.fit(
                    X_train=X_train,
                    y_train=y_train,
                    task="regression",  # 回帰問題
                    time_budget=time_budget,
                    metric="mae",  # Mean Absolute Error
                    estimator_list=['lgbm', 'xgboost', 'rf', 'extra_tree'],
                    n_splits=3,
                    verbose=0,
                    seed=42
                )
                
                # 予測ランク
                pred_ranks = automl.predict(X_test)
                
                # 評価指標
                mae = mean_absolute_error(
                    test_data.groupby('date')['current_diff'].rank(ascending=False, method='first'),
                    pred_ranks
                )
                
                # TOP3 Precision計算
                # 予測ランクが小さい順（上位）に3つ選ぶ
                predicted_top3 = [
                    d for d, _ in sorted(
                        zip(test_data['last_digit'].values, pred_ranks),
                        key=lambda x: x[1]  # ランクが小さい=上位
                    )[:3]
                ]
                actual_top3 = test_data.nlargest(3, 'current_diff')['last_digit'].tolist()
                overlap = len(set(predicted_top3) & set(actual_top3))
                precision = overlap / 3
                
                print(f"  モデル: {automl.best_estimator}")
                print(f"  MAE（ランク誤差）: {mae:.2f}")
                print(f"  Precision@3: {precision:.3f}")
                print(f"  予測: {predicted_top3} / 実際: {actual_top3}")
                
                # 予測ランク詳細
                rank_df = pd.DataFrame({
                    'digit': test_data['last_digit'].values,
                    'pred_rank': pred_ranks,
                    'actual_diff': test_data['current_diff'].values
                }).sort_values('pred_rank')
                print(f"\n  予測TOP5:")
                for idx, row in rank_df.head(5).iterrows():
                    print(f"    {row['digit']}: 予測ランク{row['pred_rank']:.1f} (実差枚{row['actual_diff']:.0f})")
                
                # 特徴量重要度
                try:
                    if hasattr(automl.model.estimator, 'feature_importances_'):
                        imp = automl.model.estimator.feature_importances_
                        top_features = pd.Series(imp, index=feature_cols).nlargest(10)
                        print(f"\n  重要特徴量TOP10:")
                        for feat, val in top_features.items():
                            print(f"    {feat}: {val:.4f}")
                except:
                    pass
                
                results.append({
                    'model_name': model_name,
                    'event': event_type,
                    'test_date': test_date,
                    'precision': precision,
                    'mae': mae,
                    'best_model': automl.best_estimator,
                    'n_features': X_train.shape[1],
                    'predicted': predicted_top3,
                    'actual': actual_top3
                })
                
            except Exception as e:
                print(f"  エラー: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    return pd.DataFrame(results)

# 実行
print("="*100)
print("ランク予測（回帰）実験")
print("="*100)

rank_results = []
for name in ['all', 'jug']:
    if name in enhanced_histories and len(enhanced_histories[name]) > 0:
        result = run_rank_regression(
            enhanced_histories[name], 
            name.upper(), 
            event_types=None,
            n_test=3,
            time_budget=180
        )
        if result is not None and len(result) > 0:
            rank_results.append(result)

# サマリー
if rank_results:
    final_rank = pd.concat(rank_results, ignore_index=True)
    
    print("\n" + "="*100)
    print("【ランク予測結果サマリー】")
    print("="*100)
    
    # モデル別平均
    for model_name in final_rank['model_name'].unique():
        subset = final_rank[final_rank['model_name'] == model_name]
        print(f"\n{model_name}:")
        print(f"  平均Precision@3: {subset['precision'].mean():.3f}")
        print(f"  平均MAE: {subset['mae'].mean():.2f} (完璧=0, ランダム≈3.6)")
        print(f"  最高Precision@3: {subset['precision'].max():.3f}")
        print(f"  使用モデル: {subset['best_model'].value_counts().to_dict()}")
    
    # 詳細結果
    print("\n【詳細結果】")
    display_cols = ['model_name', 'event', 'precision', 'mae', 'best_model']
    print(final_rank[display_cols].to_string(index=False))
    
    # ベスト結果
    best_idx = final_rank['precision'].idxmax()
    best = final_rank.loc[best_idx]
    print(f"\n【ベスト結果】")
    print(f"モデル: {best['model_name']} / イベント: {best['event']}")
    print(f"Precision@3: {best['precision']:.3f}")
    print(f"MAE: {best['mae']:.2f}")
    print(f"予測: {best['predicted']} / 実際: {best['actual']}")

print("\n✅ ランク予測完了")

In [ ]:
# ============================================================
# セルA: 環境セットアップ + データ準備
# ============================================================

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 機械学習
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# 設定
DB_PATH = 'pachinko_analysis_マルハンメガシティ柏.db'
plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# イベント定義
EVENT_DEFINITIONS = {
    'is_1day': '1day', 'is_2day': '2day', 'is_3day': '3day',
    'is_4day': '4day', 'is_5day': '5day', 'is_6day': '6day',
    'is_7day': '7day', 'is_8day': '8day', 'is_9day': '9day',
    'is_0day': '0day', 'is_39day': '39day', 'is_40day': '40day',
    'is_zorome': 'Zorome', 'is_saturday': 'Saturday', 'is_sunday': 'Sunday'
}

DIGIT_ORDER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ゾロ目']

print("✅ 環境セットアップ完了")


# ============================================================
# セルB: データ読み込み
# ============================================================

def load_last_digit_data(db_path, table_name='last_digit_summary_all'):
    """last_digit_summaryテーブル読み込み"""
    conn = sqlite3.connect(db_path)
    
    # イベントカレンダー
    df_events = pd.read_sql_query("SELECT * FROM event_calendar ORDER BY date", conn)
    
    # 末尾データ
    df = pd.read_sql_query(f"SELECT * FROM {table_name} ORDER BY date, last_digit", conn)
    df = df.merge(df_events, on='date', how='left')
    
    # 日付処理
    df['date_obj'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df['weekday'] = df['date_obj'].dt.day_name()
    df['weekday_num'] = df['date_obj'].dt.weekday
    df['day_of_month'] = df['date_obj'].dt.day
    df['is_saturday'] = (df['weekday_num'] == 5).astype(int)
    df['is_sunday'] = (df['weekday_num'] == 6).astype(int)
    
    # 末尾数値化（カテゴリ型変換前に実施）
    df['digit_num'] = df['last_digit'].apply(lambda x: 10 if x == 'ゾロ目' else int(x))
    
    # カテゴリ型に変換（数値演算後）
    df['last_digit'] = pd.Categorical(df['last_digit'], categories=DIGIT_ORDER, ordered=True)
    
    conn.close()
    
    print(f"データ読込: {len(df)} rows, {df['date'].nunique()} days, {df['last_digit'].nunique()} digits")
    return df

# 3モデル分読込
df_all = load_last_digit_data(DB_PATH, 'last_digit_summary_all')
df_jug = load_last_digit_data(DB_PATH, 'last_digit_summary_jug')
df_non_jug = load_last_digit_data(DB_PATH, 'last_digit_summary_other')

print("\n✅ データ読み込み完了")


# ============================================================
# セルC: イベント履歴ベース特徴量生成（prev_方式）
# ============================================================

def create_event_history_features(df):
    """イベント×末尾ごとの履歴特徴量（prev_方式）"""
    all_dates = sorted(df['date'].unique())
    
    print("  イベント×末尾インデックス構築中...")
    event_digit_index = {}
    
    # イベント×末尾の出現日リストを作成
    for date in all_dates:
        date_mask = df['date'] == date
        date_data = df[date_mask].iloc[0]
        
        # アクティブイベント特定
        active_events = []
        for col, label in EVENT_DEFINITIONS.items():
            if col in df.columns and date_data.get(col, 0) == 1:
                active_events.append(label.lower())
        
        # 各末尾×イベントの組み合わせに日付を追加
        for digit_num in df[date_mask]['digit_num'].unique():
            for event_name in active_events:
                key = (event_name, digit_num)
                if key not in event_digit_index:
                    event_digit_index[key] = []
                event_digit_index[key].append(date)
    
    print(f"  インデックス完成: {len(event_digit_index)} 組み合わせ")
    
    # キャッシュ作成
    data_cache = {}
    for col in ['avg_diff_coins', 'avg_games', 'win_rate', 'last_digit_rank_diff', 
                'last_digit_rank_games', 'high_profit_rate']:
        if col in df.columns:
            data_cache[col] = df.set_index(['date', 'digit_num'])[col].to_dict()
    
    print("  prev_特徴量生成中...")
    history_features = []
    
    for (event_name, digit_num), dates in event_digit_index.items():
        for i, date in enumerate(dates):
            # 当日データ
            current_data = {}
            for col, cache in data_cache.items():
                current_data[col] = cache.get((date, digit_num))
            
            if current_data['avg_diff_coins'] is None:
                continue
            
            # prev_1, prev_2, prev_3（同一イベント×末尾の過去）
            features = {
                'date': date,
                'digit_num': digit_num,
                'event_type': event_name,
                'current_diff': current_data['avg_diff_coins']
            }
            
            # 過去3回分
            for j in range(1, 4):
                if i >= j:
                    prev_date = dates[i - j]
                    for col, cache in data_cache.items():
                        val = cache.get((prev_date, digit_num))
                        features[f'prev_{j}_{col}'] = val if val is not None else 0
                else:
                    for col in data_cache.keys():
                        features[f'prev_{j}_{col}'] = 0
            
            # 変化量（prev_1との差）
            if i >= 1:
                for col in ['avg_diff_coins', 'avg_games']:
                    if col in data_cache:
                        prev_val = data_cache[col].get((dates[i-1], digit_num), 0)
                        curr_val = current_data.get(col, 0)
                        features[f'prev_1_{col}_change'] = curr_val - prev_val if prev_val else 0
            
            # 過去N回の統計量
            for window in [3, 5]:
                if i >= window:
                    past_dates = dates[max(0, i-window):i]
                    past_diffs = [data_cache['avg_diff_coins'].get((d, digit_num), 0) for d in past_dates]
                    past_diffs = [v for v in past_diffs if v is not None]
                    
                    if len(past_diffs) > 0:
                        features[f'prev_{window}_mean_diff'] = np.mean(past_diffs)
                        features[f'prev_{window}_max_diff'] = np.max(past_diffs)
                        features[f'prev_{window}_std_diff'] = np.std(past_diffs) if len(past_diffs) > 1 else 0
                    
                    # ランク統計
                    if 'last_digit_rank_diff' in data_cache:
                        past_ranks = [data_cache['last_digit_rank_diff'].get((d, digit_num), 0) for d in past_dates]
                        past_ranks = [v for v in past_ranks if v is not None and v > 0]
                        
                        if len(past_ranks) > 0:
                            features[f'prev_{window}_mean_rank'] = np.mean(past_ranks)
                            features[f'prev_{window}_top3_rate'] = sum(1 for r in past_ranks if r <= 3) / len(past_ranks)
            
            history_features.append(features)
    
    return pd.DataFrame(history_features)


def create_comprehensive_features_v2(df):
    """イベント履歴ベース + 安全な全日付特徴量"""
    df = df.copy()
    
    print("特徴量生成開始（v2: prev_方式）...")
    
    # ===== 1. 全日付でのラグ特徴量（直近N日） =====
    print("  1. 全日付ラグ特徴量...")
    for lag in [1, 7, 14]:
        for col in ['avg_diff_coins', 'avg_games', 'win_rate']:
            if col in df.columns:
                df[f'allday_{col}_lag{lag}'] = df.groupby('digit_num')[col].shift(lag)
    
    # ===== 2. 曜日×末尾交互作用 =====
    print("  2. 曜日交互作用...")
    for wd in range(7):
        df[f'is_weekday{wd}'] = (df['weekday_num'] == wd).astype(int)
        df[f'weekday{wd}_x_digit'] = df[f'is_weekday{wd}'] * df['digit_num']
    
    # ===== 3. イベント×末尾マッチング =====
    print("  3. イベントマッチング...")
    event_digit_map = {
        '1day': [1], '2day': [2], '3day': [3], '4day': [4], '5day': [5],
        '6day': [6], '7day': [7], '8day': [8], '9day': [9], '0day': [0],
        '39day': [3, 9], '40day': [4, 0], 'zorome': [10]
    }
    
    for col, label in EVENT_DEFINITIONS.items():
        if col in df.columns:
            label_lower = label.lower()
            if label_lower in event_digit_map:
                target_digits = event_digit_map[label_lower]
                df[f'match_{label_lower}'] = (
                    (df[col] == 1) & (df['digit_num'].isin(target_digits))
                ).astype(int)
    
    # ===== 4. 時系列位置 =====
    print("  4. 時系列位置...")
    df['days_since_start'] = (df['date_obj'] - df['date_obj'].min()).dt.days
    df['days_to_end'] = (df['date_obj'].max() - df['date_obj']).dt.days
    
    print(f"  基本特徴量生成完了")
    return df


print("イベント履歴特徴量関数定義完了（prev_方式）")


# ============================================================
# セルD: イベント履歴とマージ
# ============================================================

def merge_event_history_features(df_base, df_history):
    """基本特徴量とイベント履歴をマージ"""
    print("  イベント履歴マージ中...")
    
    # date, digit_num, event_typeでマージ
    merged_data = []
    
    for _, row in df_base.iterrows():
        date = row['date']
        digit_num = row['digit_num']
        
        # この行のアクティブイベント
        active_events = []
        for col, label in EVENT_DEFINITIONS.items():
            if col in df_base.columns and row.get(col, 0) == 1:
                active_events.append(label.lower())
        
        # 各イベントの履歴特徴量を取得
        for event_name in active_events:
            history_row = df_history[
                (df_history['date'] == date) &
                (df_history['digit_num'] == digit_num) &
                (df_history['event_type'] == event_name)
            ]
            
            if len(history_row) > 0:
                # 基本特徴量 + イベント履歴特徴量
                merged_row = row.to_dict()
                merged_row['event_type'] = event_name
                
                # prev_特徴量を追加
                for col in history_row.columns:
                    if col.startswith('prev_') or col in ['current_diff']:
                        merged_row[col] = history_row.iloc[0][col]
                
                merged_data.append(merged_row)
    
    result = pd.DataFrame(merged_data)
    print(f"  マージ完了: {len(result)} rows")
    return result


    print("マージ関数定義完了")
    print("  1. ラグ特徴量...")
    for lag in [1, 2, 3, 7, 14, 21, 28]:
        for col in ['avg_diff_coins', 'avg_games', 'win_rate', 'last_digit_rank_diff', 'last_digit_rank_games']:
            if col in df.columns:
                # digit_numでグループ化
                df[f'{col}_lag{lag}'] = df.groupby('digit_num')[col].shift(lag)
    
    # ===== 2. 移動統計量 =====
    print("  2. 移動統計量...")
    for window in [3, 7, 14, 21, 28]:
        for col in ['avg_diff_coins', 'avg_games', 'last_digit_rank_diff']:
            if col in df.columns:
                # digit_numでグループ化
                grouped = df.groupby('digit_num')[col]
                df[f'{col}_ma{window}'] = grouped.transform(lambda x: x.rolling(window, min_periods=1).mean())
                df[f'{col}_std{window}'] = grouped.transform(lambda x: x.rolling(window, min_periods=1).std())
                df[f'{col}_min{window}'] = grouped.transform(lambda x: x.rolling(window, min_periods=1).min())
                df[f'{col}_max{window}'] = grouped.transform(lambda x: x.rolling(window, min_periods=1).max())
    
    # ===== 3. 変化率特徴量 =====
    print("  3. 変化率...")
    for lag in [1, 7, 14]:
        for col in ['avg_diff_coins', 'avg_games']:
            if col in df.columns and f'{col}_lag{lag}' in df.columns:
                df[f'{col}_change{lag}'] = df[col] - df[f'{col}_lag{lag}']
                df[f'{col}_pct_change{lag}'] = df[col] / (df[f'{col}_lag{lag}'].replace(0, 1))
    
    # ===== 4. ランク関連特徴量 =====
    print("  4. ランク特徴量...")
    if 'last_digit_rank_diff' in df.columns:
        # TOP3率（過去N日間）- digit_numでグループ化
        for window in [7, 14, 28]:
            df[f'top3_rate_{window}d'] = df.groupby('digit_num')['last_digit_rank_diff'].transform(
                lambda x: x.rolling(window, min_periods=1).apply(lambda r: (r <= 3).mean())
            )
        
        # 最高ランク（過去N日間）
        for window in [7, 14, 28]:
            df[f'best_rank_{window}d'] = df.groupby('digit_num')['last_digit_rank_diff'].transform(
                lambda x: x.rolling(window, min_periods=1).min()
            )
    
    # ===== 5. 曜日×末尾 交互作用 =====
    print("  5. 曜日交互作用...")
    for wd in range(7):
        df[f'is_weekday{wd}'] = (df['weekday_num'] == wd).astype(int)
        df[f'weekday{wd}_x_digit'] = df[f'is_weekday{wd}'] * df['digit_num']
    
    # ===== 6. イベント×末尾 マッチング =====
    print("  6. イベントマッチング...")
    event_digit_map = {
        '1day': [1], '2day': [2], '3day': [3], '4day': [4], '5day': [5],
        '6day': [6], '7day': [7], '8day': [8], '9day': [9], '0day': [0],
        '39day': [3, 9], '40day': [4, 0], 'zorome': [10]
    }
    
    for col, label in EVENT_DEFINITIONS.items():
        if col in df.columns:
            label_lower = label.lower()
            if label_lower in event_digit_map:
                target_digits = event_digit_map[label_lower]
                # digit_numを使用（数値型）
                df[f'match_{label_lower}'] = (
                    (df[col] == 1) & (df['digit_num'].isin(target_digits))
                ).astype(int)
    
    # ===== 7. イベント履歴特徴量 =====
    print("  7. イベント履歴...")
    # 各イベントでの過去実績
    for col, label in EVENT_DEFINITIONS.items():
        if col in df.columns:
            event_mask = df[col] == 1
            event_data = df[event_mask].copy()
            
            if len(event_data) > 0:
                # last_digitを文字列に変換してgroupby
                event_data['digit_str'] = event_data['digit_num'].astype(str)
                
                # イベント時の平均差枚（末尾別）- expanding mean
                event_avg_dict = {}
                for digit_str in event_data['digit_str'].unique():
                    digit_data = event_data[event_data['digit_str'] == digit_str]['avg_diff_coins']
                    expanding_mean = digit_data.expanding(min_periods=1).mean()
                    event_avg_dict[digit_str] = expanding_mean
                
                # DataFrameに格納
                df[f'{label.lower()}_hist_avg'] = 0.0
                for digit_str, expanding_mean in event_avg_dict.items():
                    mask = event_mask & (df['digit_num'] == int(digit_str))
                    df.loc[mask, f'{label.lower()}_hist_avg'] = expanding_mean.values
                
                # 前方埋め（次回イベント予測用）- digit_numでグループ化
                df[f'{label.lower()}_hist_avg'] = df.groupby('digit_num')[f'{label.lower()}_hist_avg'].ffill()
    
    # ===== 8. 組み合わせ特徴量 =====
    print("  8. 組み合わせ...")
    if 'avg_diff_coins' in df.columns and 'avg_games' in df.columns:
        df['efficiency'] = df['avg_diff_coins'] / df['avg_games'].replace(0, 1)
        # digit_numでグループ化
        df['efficiency_lag1'] = df.groupby('digit_num')['efficiency'].shift(1)
        df['efficiency_ma7'] = df.groupby('digit_num')['efficiency'].transform(
            lambda x: x.rolling(7, min_periods=1).mean()
        )
    
    # ===== 9. 時系列位置特徴量 =====
    print("  9. 時系列位置...")
    df['days_since_start'] = (df['date_obj'] - df['date_obj'].min()).dt.days
    df['days_to_end'] = (df['date_obj'].max() - df['date_obj']).dt.days
    
    # ===== 10. 統計的特徴量 =====
    print("  10. 統計量...")
    for window in [7, 14]:
        if 'avg_diff_coins' in df.columns:
            # digit_numでグループ化
            grouped = df.groupby('digit_num')['avg_diff_coins']
            
            # 歪度
            skew_col = f'diff_skew{window}'
            df[skew_col] = grouped.transform(
                lambda x: x.rolling(window, min_periods=3).skew()
            )
            
            # 尖度
            kurt_col = f'diff_kurt{window}'
            df[kurt_col] = grouped.transform(
                lambda x: x.rolling(window, min_periods=3).apply(
                    lambda y: y.kurtosis() if len(y) > 2 else 0
                )
            )
    
    print(f"  生成完了: {len([c for c in df.columns if c not in ['date', 'last_digit', 'date_obj', 'weekday']])} 特徴量")
    return df

print("網羅的特徴量生成関数定義完了")


# ============================================================
# セルE: イベント別データ分割 + 重要度評価
# ============================================================

def prepare_event_data(df_merged, event_name, n_test=3):
    """イベント別データ準備（prev_特徴量版）"""
    event_data = df_merged[df_merged['event_type'] == event_name].copy()
    event_dates = sorted(event_data['date'].unique())
    
    if len(event_dates) < n_test + 5:
        return None, None, None, None, None, None
    
    # 訓練：最後のn_test日を除く全て
    # テスト：最後のn_test日
    train_dates = event_dates[:-n_test]
    test_dates = event_dates[-n_test:]
    
    train_data = event_data[event_data['date'].isin(train_dates)].copy()
    test_data = event_data[event_data['date'].isin(test_dates)].copy()
    
    # TOP3ラベル作成
    train_data['rank'] = train_data.groupby('date')['current_diff'].rank(ascending=False, method='first')
    train_data['is_top3'] = (train_data['rank'] <= 3).astype(int)
    
    test_data['rank'] = test_data.groupby('date')['current_diff'].rank(ascending=False, method='first')
    test_data['is_top3'] = (test_data['rank'] <= 3).astype(int)
    
    # 🔴 prev_特徴量のみ使用（完全リーク防止）
    safe_patterns = [
        'prev_',  # イベント履歴特徴量
        'allday_',  # 全日付ラグ
        'weekday', 'day_of_month', 'is_saturday', 'is_sunday',
        'days_since_start', 'days_to_end',
        'is_1day', 'is_2day', 'is_3day', 'is_4day', 'is_5day',
        'is_6day', 'is_7day', 'is_8day', 'is_9day', 'is_0day',
        'is_39day', 'is_40day', 'is_zorome',
        'match_', 'digit_num'
    ]
    
    # メタ情報除外
    exclude_cols = ['date', 'date_obj', 'last_digit', 'weekday', 'event_type',
                   'current_diff', 'rank', 'is_top3']
    
    feature_cols = []
    for c in train_data.columns:
        if c in exclude_cols:
            continue
        if train_data[c].dtype not in ['int64', 'float64']:
            continue
        if any(pattern in c for pattern in safe_patterns):
            feature_cols.append(c)
    
    print(f"  使用特徴量: {len(feature_cols)} 個")
    
    # prev_特徴量の内訳表示
    prev_features = [c for c in feature_cols if c.startswith('prev_')]
    other_features = [c for c in feature_cols if not c.startswith('prev_')]
    print(f"    - prev_特徴量: {len(prev_features)}")
    print(f"    - その他: {len(other_features)}")
    
    X_train = train_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    y_train = train_data['is_top3'].values
    X_test = test_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    y_test = test_data['is_top3'].values
    
    return X_train, y_train, X_test, y_test, test_data, feature_cols, leak_keywords
        
    # 安全パターンに該当するか
    is_safe = any(pattern in c for pattern in safe_patterns)
        
    if not is_leak and is_safe:
        feature_cols.append(c)
    else:
        excluded_features.append(c)
    
    print(f"  全カラム: {len(train_data.columns)}")
    print(f"  使用特徴量: {len(feature_cols)} 個")
    print(f"  除外: {len(excluded_features)} 個")
    
    # 除外された重要そうな特徴量を表示
    if excluded_features:
        print(f"  除外例: {excluded_features[:15]}")
    
    X_train = train_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    y_train = train_data['is_top3'].values
    X_test = test_data[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    y_test = test_data['is_top3'].values
    
    return X_train, y_train, X_test, y_test, test_data, feature_cols


def evaluate_feature_importance(df_merged, event_name):
    """特徴量重要度評価（3モデル比較）- prev_方式"""
    print(f"\n{'='*80}")
    print(f"【{event_name.upper()}】重要度評価")
    print(f"{'='*80}")
    
    # データ準備
    result = prepare_event_data(df_merged, event_name, n_test=3)
    if result[0] is None:
        print(f"データ不足: {event_name}")
        return None
    
    X_train, y_train, X_test, y_test, test_data, feature_cols = result
    
    print(f"訓練: {len(X_train)} samples × {len(feature_cols)} features")
    print(f"テスト: {len(X_test)} samples")
    print(f"TOP3比率: train={y_train.mean():.3f}, test={y_test.mean():.3f}")
    
    # モデル訓練
    models = {
        'RF': RandomForestClassifier(
            n_estimators=200, max_depth=10, min_samples_split=5,
            random_state=42, n_jobs=-1, class_weight='balanced'
        ),
        'XGB': XGBClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, eval_metric='logloss'
        ),
        'LGBM': LGBMClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, verbose=-1, class_weight='balanced'
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\n--- {name} ---")
        model.fit(X_train, y_train)
        
        # 予測
        if hasattr(model, 'predict_proba'):
            proba = model.predict_proba(X_test)[:, 1]
        else:
            proba = model.predict(X_test)
        
        # Precision@3計算（日別）
        precisions = []
        for date in test_data['date'].unique():
            date_mask = test_data['date'] == date
            date_proba = proba[date_mask]
            date_digits = test_data[date_mask]['last_digit'].values
            date_actual = test_data[date_mask]['current_diff'].values
            
            # 予測TOP3
            pred_top3 = [d for d, _ in sorted(zip(date_digits, date_proba), 
                                              key=lambda x: x[1], reverse=True)[:3]]
            # 実際TOP3
            actual_top3 = [d for d, _ in sorted(zip(date_digits, date_actual),
                                                key=lambda x: x[1], reverse=True)[:3]]
            
            overlap = len(set(pred_top3) & set(actual_top3))
            precisions.append(overlap / 3)
        
        avg_precision = np.mean(precisions)
        print(f"Precision@3: {avg_precision:.3f}")
        
        # 特徴量重要度
        importances = model.feature_importances_
        importance_df = pd.DataFrame({
            'feature': feature_cols,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        results[name] = {
            'model': model,
            'precision': avg_precision,
            'importance': importance_df,
            'test_proba': proba,
            'test_data': test_data
        }
    
    return results


# ============================================================
# セルF: 全イベント一括評価（prev_方式）
# ============================================================

def run_comprehensive_evaluation(df, model_name='ALL'):
    """全イベントで重要度評価実行（prev_方式）"""
    print(f"\n{'='*100}")
    print(f"【{model_name}】包括的評価開始（prev_方式）")
    print(f"{'='*100}")
    
    # 1. 基本特徴量生成
    print("\n基本特徴量生成中...")
    df_base = create_comprehensive_features_v2(df)
    
    # 2. イベント履歴特徴量生成
    print("\nイベント履歴特徴量生成中...")
    df_history = create_event_history_features(df)
    
    print(f"\nイベント履歴データ: {len(df_history)} rows")
    print(f"ユニークイベント: {df_history['event_type'].nunique()}")
    
    # 3. マージ
    print("\n特徴量マージ中...")
    df_merged = merge_event_history_features(df_base, df_history)
    
    print(f"マージ後: {len(df_merged)} rows")
    print(f"特徴量数: {len(df_merged.columns)}")
    
    # 4. 評価対象イベント（データ量上位5個）
    event_counts = df_merged['event_type'].value_counts().head(5)
    
    print(f"\n評価対象: {len(event_counts)} イベント")
    for event_name, count in event_counts.items():
        print(f"  {event_name}: {count} rows")
    
    # 5. 各イベントで評価
    all_results = {}
    
    for event_name in event_counts.index:
        results = evaluate_feature_importance(df_merged, event_name)
        if results:
            all_results[event_name] = results
    
    return all_results, df_merged


# ============================================================
# セルG: 重要度可視化
# ============================================================

def visualize_importance(all_results, top_n=30):
    """特徴量重要度可視化"""
    for event_name, results in all_results.items():
        print(f"\n{'='*100}")
        print(f"【{event_name}】重要特徴量TOP{top_n}")
        print(f"{'='*100}\n")
        
        # 3モデル比較
        fig, axes = plt.subplots(1, 3, figsize=(20, 8))
        
        for idx, (model_name, result) in enumerate(results.items()):
            importance_df = result['importance'].head(top_n)
            
            ax = axes[idx]
            ax.barh(range(len(importance_df)), importance_df['importance'].values)
            ax.set_yticks(range(len(importance_df)))
            ax.set_yticklabels(importance_df['feature'].values, fontsize=8)
            ax.invert_yaxis()
            ax.set_xlabel('Importance', fontsize=10)
            ax.set_title(f'{model_name} (Precision@3: {result["precision"]:.3f})', 
                        fontsize=12, fontweight='bold')
            ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # テキストレポート
        print(f"\n【モデル別Precision@3】")
        for model_name, result in results.items():
            print(f"{model_name}: {result['precision']:.3f}")
        
        print(f"\n【共通重要特徴量TOP15】")
        # 3モデルの順位を平均
        all_features = set()
        for result in results.values():
            all_features.update(result['importance']['feature'].head(30).tolist())
        
        feature_ranks = {f: [] for f in all_features}
        for result in results.values():
            imp_dict = result['importance'].set_index('feature')['importance'].to_dict()
            for f in all_features:
                feature_ranks[f].append(imp_dict.get(f, 0))
        
        avg_importance = {f: np.mean(ranks) for f, ranks in feature_ranks.items()}
        sorted_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)
        
        for rank, (feat, imp) in enumerate(sorted_features[:15], 1):
            print(f"{rank:2d}. {feat:50s} {imp:.6f}")


# ============================================================
# セルH: 実行
# ============================================================

print("="*100)
print("prev_方式 特徴量重要度評価 開始")
print("="*100)

# 実行（ALLモデルのみ、時間短縮のため）
all_results, df_merged = run_comprehensive_evaluation(df_all, 'ALL')

# 重要度可視化
visualize_importance(all_results, top_n=30)

print("\n✅ prev_方式 特徴量重要度評価完了")